In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# dummy image
img_size = (32, 32, 3)
img = np.random.randint(0, 255, img_size)
# img_expanded = np.expand_dims(img, axis=0)
processed_img = tf.keras.applications.mobilenet.preprocess_input(img)

In [ ]:
# model
model = tf.keras.applications.mobilenet.MobileNet(input_shape = processed_img.shape, classes=10, include_top=False)


In [ ]:
# get summary statics
pred = model.predict(processed_img)
model.summary()
n_params = model.count_params()
print(f"Image shape:{processed_img.shape}, No. of params:{n_params}")